In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys, os, pickle, utils

from tqdm.notebook import tqdm
from datetime import timedelta
#from utils import baseline_SCr

if os.getcwd()[-4:] == "code":
    os.chdir('../')

icu = './data/mimic-iv-2.2-parquet/icu/'
hosp = './data/mimic-iv-2.2-parquet/hosp/'

pd.set_option('mode.chained_assignment',  None) # 경고 off

In [25]:
df = pd.read_parquet('./data/resample/resample_merge.parquet')
len(df.stay_id.unique())

73181

In [26]:
df.sort_values(by=['subject_id','hadm_id','stay_id','charttime'],inplace=True)
df['step'] = df.groupby('stay_id').cumcount()

In [27]:
# 24시간 이내 퇴원
df = df.groupby('stay_id').filter(lambda x:len(x) > 24)
len(df.stay_id.unique())

57734

In [10]:
# 24시간 이내 AKI 발생

# 각 stay_id에 대해 처음 24개 행을 선택
first_24 = df.groupby('stay_id').head(24)[['step','subject_id','hadm_id','stay_id','charttime','uo','AKI_UO','SCr','baseline_SCr','AKI_SCr','AKI']]

# 'AKI' 지표가 1 이상인 행을 포함하는 stay_id를 제외
filtered_df = first_24.groupby('stay_id').filter(lambda x: not (x['AKI'] >= 1).any())

# 원래 DataFrame에서 필터링된 stay_id만 선택
df = df[df['stay_id'].isin(filtered_df['stay_id'].unique())]

len(df.stay_id.unique())

15759

In [11]:
# 각 stay_id별로 count_category가 3 이상인 경우의 개수 계산
count_3_or_more = df.groupby('stay_id')['count_category'].transform(lambda x: (x >= 3).sum())

# count_category가 3 이상인 경우가 없는 stay_id만 필터링
df = df[count_3_or_more < 1]
len(df.stay_id.unique())

15540

In [12]:
df.count_category.value_counts()

count_category
0.0    1270869
1.0     104551
2.0       6759
Name: count, dtype: int64

In [24]:
tm = df2.isna().sum()

In [23]:
df2 = utils.resample_fill(df)

In [22]:
df[df['baseline_SCr'].isna()]

,subject_id,hadm_id,stay_id,charttime,HR,SBP,DBP,MAP,temp,RR,...,Cephalosporins,Vancomycin,Betalactam_comb,Metronidazole,Carbapenems,Penicillins,Fluoroquinolones,Others,count_category,step
3326504,15474523,20266641,37084601,2170-10-23 20:56:16,93.0,185.0,61.0,102.333333,37.2,26.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3326505,15474523,20266641,37084601,2170-10-23 21:56:16,84.0,147.0,52.0,83.666667,37.2,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3326506,15474523,20266641,37084601,2170-10-23 22:56:16,78.0,127.0,44.0,71.666667,37.2,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3326507,15474523,20266641,37084601,2170-10-23 23:56:16,70.0,135.0,42.0,73.000000,36.5,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3326508,15474523,20266641,37084601,2170-10-24 00:56:16,86.0,139.0,50.0,79.666667,36.5,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3326569,15474523,20266641,37084601,2170-10-26 13:56:16,65.0,137.0,40.0,72.333333,36.1,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65
3326570,15474523,20266641,37084601,2170-10-26 14:56:16,80.0,135.0,46.0,75.666667,36.7,15.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,66
3326571,15474523,20266641,37084601,2170-10-26 15:56:16,84.0,135.0,46.0,75.666667,36.7,38.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67
3326572,15474523,20266641,37084601,2170-10-26 16:56:16,78.0,135.0,46.0,75.666667,36.7,38.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,68
